First step is to call the api and check we can receive responses.

In [31]:
import requests as rq

response = rq.get("https://environment.data.gov.uk/flood-monitoring/id/stations/531160.json")

print(response.status_code)

print(response.json())

200
{'@context': 'http://environment.data.gov.uk/flood-monitoring/meta/context.jsonld', 'meta': {'publisher': 'Environment Agency', 'licence': 'http://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/', 'documentation': 'http://environment.data.gov.uk/flood-monitoring/doc/reference', 'version': '0.9', 'comment': 'Status: Beta service', 'hasFormat': ['http://environment.data.gov.uk/flood-monitoring/id/stations/531160.rdf', 'http://environment.data.gov.uk/flood-monitoring/id/stations/531160.ttl', 'http://environment.data.gov.uk/flood-monitoring/id/stations/531160.html']}, 'items': {'@id': 'http://environment.data.gov.uk/flood-monitoring/id/stations/531160', 'RLOIid': '3416', 'catchmentName': 'Bristol Avon Little Avon Axe and North Somerset St', 'dateOpened': '2008-12-01', 'datumOffset': 4.115, 'eaAreaName': 'South West - Wessex', 'eaRegionName': 'South West', 'easting': 366165, 'gridReference': 'ST6616569003', 'label': 'Keynsham, Rivermead', 'lat': 51.419019, 'long': -2.

Next format the JSON into something nicer with json.dumps

In [32]:
import requests as rq
import json

## define function jprint which makes the json output look pretty and easy to understand
def jprint(obj):
        text = json.dumps(obj, sort_keys=True, indent=4)
        print(text)

response = rq.get("https://environment.data.gov.uk/flood-monitoring/id/measures/531160-level-stage-i-15_min-mASD.json") ##updated with the measure api instead of the station one

print(response.status_code)

jprint(response.json())

200
{
    "@context": "http://environment.data.gov.uk/flood-monitoring/meta/context.jsonld",
    "items": {
        "@id": "http://environment.data.gov.uk/flood-monitoring/id/measures/531160-level-stage-i-15_min-mASD",
        "datumType": "http://environment.data.gov.uk/flood-monitoring/def/core/datumASD",
        "label": "KEYNSHAM ADCP - level-stage-i-15_min-mASD",
        "latestReading": {
            "@id": "http://environment.data.gov.uk/flood-monitoring/data/readings/531160-level-stage-i-15_min-mASD/2024-02-29T00-00-00Z",
            "date": "2024-02-29",
            "dateTime": "2024-02-29T00:00:00Z",
            "measure": "http://environment.data.gov.uk/flood-monitoring/id/measures/531160-level-stage-i-15_min-mASD",
            "value": 5.141
        },
        "notation": "531160-level-stage-i-15_min-mASD",
        "parameter": "level",
        "parameterName": "Water Level",
        "period": 900,
        "qualifier": "Stage",
        "station": "http://environment.data.go

trying to get just the particular object

In [43]:
import requests as rq
import json

## define function jprint which makes the json output look pretty and easy to understand
def jprint(obj):
        text = json.dumps(obj['items']['latestReading'], sort_keys=True, indent=4)
        print(text)

response = rq.get("https://environment.data.gov.uk/flood-monitoring/id/measures/531160-level-stage-i-15_min-mASD.json") ##updated with the measure api instead of the station one

print(response.status_code)

jprint(response.json())



200
{
    "@id": "http://environment.data.gov.uk/flood-monitoring/data/readings/531160-level-stage-i-15_min-mASD/2024-02-29T00-15-00Z",
    "date": "2024-02-29",
    "dateTime": "2024-02-29T00:15:00Z",
    "measure": "http://environment.data.gov.uk/flood-monitoring/id/measures/531160-level-stage-i-15_min-mASD",
    "value": 5.141
}


Ok with that bit done we can set a variable to the value of the api response and then print that.



In [2]:
import requests as rq
import json
# Import Gauge and start_http_server from prometheus_client
from prometheus_client import Gauge, start_http_server

## initialise the guages

response = rq.get("https://environment.data.gov.uk/flood-monitoring/id/measures/531160-level-stage-i-15_min-mASD.json")

## define function jprint which makes the json output look pretty and easy to understand
def jprint(obj):
        text = json.dumps(obj['items']['latestReading']['value'])
        text = str(text)
        print(text)

height = jprint(response.json())







5.142


Now try to put it all together so that we can send the information to promoetheus

In [1]:
import requests as rq
import json
import time
# Import Gauge and start_http_server from prometheus_client
from prometheus_client import Gauge, start_http_server

## set read interval
READ_INTERVAL = 15

## set read interval units. 1 = seconds, 60 = minutes, 3600 = hours, 86400 = days
READ_UNITS = 1

## initialise the gauges

gauge_river_level = Gauge('keynsham_river_level', 'River level at Keynsham Rivermeads')

## define function jprint which makes the json output look pretty and easy to understand
def jprint(obj):
        height = json.dumps(obj['items']['latestReading']['value'])
        return float(height)
        

## define function getriver height from api
def getheight():
        ## get response
        response = rq.get("https://environment.data.gov.uk/flood-monitoring/id/measures/531160-level-stage-i-15_min-mASD.json")
        ## set river guage river level to output of jprint function
        gauge_river_level.set(jprint(response.json()))
        print(gauge_river_level)
        time.sleep(READ_INTERVAL * READ_UNITS)

if __name__ == "__main__":
        #expose metrics
        metrics_port = 8897
        start_http_server(metrics_port)
        print("Serving sensor metrics on :{}".format(metrics_port))

        while True:
                getheight()

Serving sensor metrics on :8897
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_level
gauge:keynsham_river_le

KeyboardInterrupt: 